In [12]:
import essentia.standard as ess
import numpy as np
from scipy.signal import convolve
import glob, os, sys


In [13]:
import sox
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddImpulseResponse

In [14]:
sys.path.append('../../')

In [22]:
ir_path = './IR/'

In [16]:
audio_path = '../Databases/Raw_database/'
raw_audios = glob.glob(audio_path + '*_DI.wav') #+ glob.glob(audio_path + '*_Reference.wav')
sample_rate = 44100
# audio = ess.MonoLoader(filename=audio_path, sampleRate=sample_rate)()

In [6]:
# Filter ir files

In [18]:
filtered_ir =['classroom','diningroom','livingroom','bedroom','office','kitchen']
not_wanted = ['conferenceroom','1txts','1txs','meetingroom','bathroom','hallway','largebrickwalledopenplanoffic','lobby','foyer']
ir_files = []
for f in glob.glob(ir_path + '*.wav'):
    f_norm = f.lower()
    if any(room in f_norm for room in filtered_ir):
        if any(room in f_norm for room in not_wanted ):
            pass
        else:
            ir_files.append(f)

In [20]:
reverb_names = [r.split('_')[0].replace(ir_path,'') for r in ir_files]

# APPLY REVERBERATIONS

In [23]:
store_path = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Reverbs/'
for aud in raw_audios:
    audio = ess.MonoLoader(filename=aud, sampleRate=sample_rate)()
    for ir,rev in zip(ir_files,reverb_names):
        ir_samples = list(ess.MonoLoader(filename=ir, sampleRate=sample_rate)())
        
        tfm = sox.Transformer()
        tfm.set_globals(guard = True) 
        tfm.fir(ir_samples)
        output_file_name = store_path + aud.split('/')[-1].replace('.wav','_') +rev + '.wav'
        
        tfm.build_file(input_array = audio,
                        sample_rate_in = sample_rate,
                        output_filepath = output_file_name)
        
        # ess.MonoWriter(filename= './Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_') +rev.replace('.wav','') + '.wav' , format='wav')(audio_convolved)

# APPLY EFFECTS

Chorus

In [28]:
store_path = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Effects/'
for aud in raw_audios:
# Chorus
    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.chorus()
    output_file_name = store_path+ aud.split('/')[-1].replace('.wav','_')
    tfm.build_file(aud, output_file_name +'Chorus'+ '.wav')

# Flanger

    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.flanger()

    tfm.build_file(aud,output_file_name +'Flanger'+ '.wav')

# Overdrive

    tfm = sox.Transformer()
    tfm.set_globals(guard = False)
    tfm.overdrive()

    tfm.build_file(aud,output_file_name +'OverDrive'+ '.wav')

# Phaser

    tfm = sox.Transformer()
    tfm.set_globals(guard = True)
    tfm.phaser()

    tfm.build_file(aud,output_file_name +'Phaser'+ '.wav')


# APPLY FILTERS

In [27]:
store_path = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/Filters/'
freq_h = [200,300,400,500]
freq_l = [2000,2500,1000,500,300]
boost = [-20,-10,0,10,20]
for audio_path in raw_audios:
    output_file = store_path+ audio_path.split('/')[-1].replace('.wav','_')
# High pass
    for f in freq_h:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.highpass(frequency = f)
        
        
        tfm.build_file(audio_path, output_file+'Hpass_'+str(f)+ '.wav')


# Low pass
    for f in freq_l:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.lowpass(frequency = f)

        tfm.build_file(audio_path, output_file +'Lpass_'+str(f)+ '.wav')


# Treble
    for b in boost:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.treble(gain_db = b)

        tfm.build_file(audio_path,output_file +'Treble_'+str(b)+ '.wav')
# Bass
    boost = [-20,20]
    for b in boost:
        tfm = sox.Transformer()
        tfm.set_globals(guard = True)
        tfm.bass(gain_db = b)

        tfm.build_file(audio_path,output_file +'Bass_'+str(b)+ '.wav')
    
    


# APPLY GAUSSIAN RANDOM NOISE

In [68]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

NOISES = 5
SAMPLE_RATE = 441000

for n in range(0,NOISES):
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),])

    # Augment/transform/perturb the audio data
    augmented_samples = augment(samples=audio, sample_rate=SAMPLE_RATE)

    ess.MonoWriter(filename= './Reverbs/'+ audio_path.split('/')[-1].replace('.wav','_')+'noise'+str(n)+'.wav', format='wav')(augmented_samples)